In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import pickle
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
import re
import jieba
from ckip_transformers.nlp import CkipWordSegmenter
import csv

2022-12-27 14:11:42.353297: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-27 14:11:42.439772: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-27 14:11:42.459898: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-27 14:11:42.821907: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li

In [2]:
course_df_original = pd.read_csv('../data/courses.csv').fillna('')
course_df = course_df_original.drop(['course_id', 'teacher_id', 'course_published_at_local', 'course_published_at_local', 'course_price'], axis=1)
course_df['description'] = course_df['description'].replace('([\<]).*?([\>])','',regex=True)
course_df = course_df.replace('\n', '',regex=True)
course_df = course_df.replace('[\d_]', '',regex=True).astype(str)
course_df.insert(0, 'course_id', course_df_original['course_id'])
course_df[:2]

,course_id,course_name,teacher_intro,groups,sub_groups,groups+subgroups,topics,description,will_learn,required_tools,recommended_background,target_group
0,61888e868f154b000781b45a,少女人妻華麗變身：七大妝容七彩的夢幻樂園,在美妝 KOL 圈裡屬個人風格強烈的 Alice，在清新與叛逆風格間遊刃有餘，其幽默的美妝影...,生活品味,"更多生活品味,護膚保養與化妝","生活品味更多生活品味,生活品味護膚保養與化妝","更多生活品味,護膚保養與化妝",少女人妻第一堂線上課程，跟著我一起華麗變身！在社群上許多人經常問我：「為什麼人妻畫的妝那麼好...,不再害怕各種顏色的彩妝，可以更隨心搭配各種繽紛的顏色。,所需工具為：視課程實際會用到的彩妝用品,只要你有一顆愛化妝、想變漂亮的心皆可以參加。⚠️ 雖然課程當中會帶到相關彩妝技巧，不過內容偏...,熱愛彩妝的人
1,54d5a117065a7e0e00725ac0,幾何圖形分割 X 色塊組合,從學生時代開始摸索photoshop等軟體，自以為有些天賦但後來發現其實沒有。出社會後從事美...,"藝術,設計","平面設計,繪畫與插畫","設計平面設計,藝術繪畫與插畫","Illustrator/以拉,配色技巧",這是一個有趣的小技巧，主要利用illustrator裡面的&quot;分割“這個功能，我已經...,可以將任何區塊分割成自己想要的幾何圖形、快速的上色，並且能夠應用在許多地方,"Adobe Illustrator（必備）, camera",知道如何使用Illustrator的基本工具列,每一位興趣的人都能學得來，非常容易的小技巧


In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import datetime
from scipy.sparse import csr_matrix

In [4]:
info = []
for i in range(course_df.shape[0]):
    t = course_df.loc[i, :].values.flatten().tolist()
    info.append([t[0], ' '.join(t)])
info = pd.DataFrame(info, columns = ['course_id', 'text'])
info[:5]

,course_id,text
0,61888e868f154b000781b45a,61888e868f154b000781b45a 少女人妻華麗變身：七大妝容七彩的夢幻樂園 ...
1,54d5a117065a7e0e00725ac0,54d5a117065a7e0e00725ac0 幾何圖形分割 X 色塊組合 從學生時代開始...
2,54d5d9952246e60a009ec571,54d5d9952246e60a009ec571 數位拼貼的手感 自由工作者，致力於品牌視覺...
3,54d7148a2246e60a009ec588,54d7148a2246e60a009ec588 Line 的貼圖自己動手做！ 我是Dann...
4,5513e92b38239d10005778e1,5513e92b38239d10005778e1 為申請學校或工作寫好英文自傳 在北美長大，...


In [5]:
info

,course_id,text
0,61888e868f154b000781b45a,61888e868f154b000781b45a 少女人妻華麗變身：七大妝容七彩的夢幻樂園 ...
1,54d5a117065a7e0e00725ac0,54d5a117065a7e0e00725ac0 幾何圖形分割 X 色塊組合 從學生時代開始...
2,54d5d9952246e60a009ec571,54d5d9952246e60a009ec571 數位拼貼的手感 自由工作者，致力於品牌視覺...
3,54d7148a2246e60a009ec588,54d7148a2246e60a009ec588 Line 的貼圖自己動手做！ 我是Dann...
4,5513e92b38239d10005778e1,5513e92b38239d10005778e1 為申請學校或工作寫好英文自傳 在北美長大，...
...,...,...
723,6191c002a20c140006edcfcd,6191c002a20c140006edcfcd Home appétit！在地食材打造道地...
724,60ee8d419b2ee60007c940cf,60ee8d419b2ee60007c940cf 偵探攻略：推理詭計X故事創作 寫作課 本來...
725,61b941f2e8990300069e4e9e,61b941f2e8990300069e4e9e 在家輕鬆做 款健康無負擔高含水歐式麵包 ...
726,6164fd2cc74856000619aa16,6164fd2cc74856000619aa16 Zunni 美甲全科班：從質感設計開始創業...


In [6]:
ws = WS('./ckip/data')

/home/ian/miniconda3/envs/adlhw3/lib/python3.8/site-packages/ckiptagger/model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
2022-12-27 14:11:47.202996: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-12-27 14:11:47.203040: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (TaiwanFarmer): /proc/driver/nvidia/version does not exist
2022-12-27 14:11:47.977741: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other o

In [7]:
#tfidf_ml = TfidfVectorizer(stop_words='english')
tfidf_ml = TfidfVectorizer(tokenizer=jieba.lcut)
tfidf_ml.fit(info.text)

# TF-IDF Dicitonary
dictionary = dict(zip(tfidf_ml.get_feature_names_out(), list(tfidf_ml.idf_)))

# feature name
tfidf_feature = tfidf_ml.get_feature_names()

/home/ian/miniconda3/envs/adlhw3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.303 seconds.
Prefix dict has been built successfully.
/home/ian/miniconda3/envs/adlhw3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [8]:
len(dictionary.keys())

73502

In [9]:
for keys,values in dictionary.items():
    print(keys)
    print(values)


2.608067110300322
	
4.036325670519244

5.982235819574558

5.982235819574558

6.898526551448713

5.026724374547121
 
1.0
!
2.608067110300322
"
3.3575672274113986
#
2.986503546020567
##
6.898526551448713
#.
6.493061443340548
$
2.8125502388971286
%
1.9676562258213197
%%
6.493061443340548
%&
6.898526551448713
%-
6.493061443340548
&
2.058284243281138
'
3.720472721100767
(
1.628094388390212
)
1.587786664902119
*
3.4645393469635666
+
2.409890181716573
+&
6.898526551448713
++
6.493061443340548
++&
6.898526551448713
,
1.4095888252920266
-
1.2763157304863233
-%
5.982235819574558
-&
5.1067670822206574
--
4.157686527523512
--&
6.493061443340548
---
5.1067670822206574
---&
6.898526551448713
----
6.2053793708887675
-----
5.799914262780603
-----%
6.898526551448713
------
6.898526551448713
-------
6.898526551448713
--------
6.898526551448713
-------------
6.898526551448713
---------------------
6.493061443340548
-----------------------------------------------------------------------------
6.49306

In [10]:
tfidf_feature[:10]

['\x08', '\t', '\x0b', '\r', '\x1b', '\x1d', ' ', '!', '"', '#']

In [11]:
# #Construct the required TF-IDF matrix by fitting and transforming the data
# vectorizer = TfidfVectorizer(stop_words= 'english')
# tfidf_matrix = vectorizer.fit_transform(info)

# #print(tfidf.vocabulary_)
# tfidf_matrix_nd = tfidf_matrix.T.toarray()
# #np.save('tfidf_embedding/item_embeddings_english.npy', tfidf_matrix_nd)
# df = pd.DataFrame(tfidf_matrix_nd, index=vectorizer.get_feature_names_out ())

In [12]:
from gensim.models.word2vec import Word2Vec

w2v_model = Word2Vec(info.text.apply(lambda text: text.split()))
w2v_vocab = list(w2v_model.wv.index_to_key)
print(w2v_model)

Word2Vec<vocab=1314, vector_size=100, alpha=0.025>


In [13]:
starttime = datetime.datetime.now()

# TF-IDF weighted Word2Vec
tfidf_text_vect = [] # tfidf-w2v is stored in this list
row = 0

for text in info.text.apply(lambda text: text.split()):
    text_vect = np.zeros(100)
    weight_sum = 0
    for word in text:
        if word in w2v_vocab and word in tfidf_feature:
            vec = w2v_model.wv[word]
            tf_idf = dictionary[word]*(text.count(word)/len(text))
            text_vect += (vec * tf_idf)
            weight_sum += tf_idf
    if weight_sum != 0:
        text_vect /= weight_sum
    tfidf_text_vect.append(text_vect)
    row += 1

# calculate running time
endtime = datetime.datetime.now()
print("建立模型時間: ",endtime - starttime)



建立模型時間:  0:00:17.972443


In [14]:
def Preprocess(text):
    rule = re.compile(r'[^a-zA-Z0-9\u4e00-\u9fa5]')
    text = rule.sub(' ',str(text))
    text = re.sub(' +', '',text)
    text = re.sub(' ', '',text)
    text = ws([text], sentence_segmentation=True)
    text = [x for l in text for x in l]
    return text

In [15]:
def recommend_law(text, tfidf_text_vect = tfidf_text_vect, top_k = 50):
    text = Preprocess(text)
    text_vect = np.zeros(100) # w2v size
    weight_sum = 0
    for word in text:
        if word in w2v_vocab and word in tfidf_feature:
            vec = w2v_model.wv[word]
            tf_idf = dictionary[word]*(text.count(word)/len(text))
            text_vect += (vec * tf_idf)
            weight_sum += tf_idf
    if weight_sum != 0:
        text_vect /= weight_sum
    tmp_vect = [*tfidf_text_vect,text_vect]
    new_cos_sim = cosine_similarity(tmp_vect, tmp_vect)
    sim_score = np.sort(new_cos_sim[new_cos_sim.shape[0]-1])[::-1][1:top_k+1]
    tmp_top_k = info[['text']].iloc[np.argsort(new_cos_sim[new_cos_sim.shape[0]-1])[::-1][1:top_k+1]]
    tmp_top_k['similarity_score'] = [round(score*100,1) for score in sim_score]
    
    return tmp_top_k



In [16]:
starttime = datetime.datetime.now()
newtext = '攝影_更多'
#newtext = ''
result = recommend_law(newtext)

# calculate running time
endtime = datetime.datetime.now()
print("搜尋推薦時間: ",endtime - starttime)
result

搜尋推薦時間:  0:00:00.195050


,text,similarity_score
20,55c636d27b4d99100011a4aa 自己當導演！全方位微電影製作（基礎） 專門...,100.0
286,5bb45dacc3964c001eed8269 火頭工做麵包：用科學方法學做健康麵包 聯經...,99.9
255,5a671c0a85b4c2001eb80ca1 興趣變副業！打造自己的手作品牌 張譯蓁，年...,99.9
674,60c2e123dcc8c38df47514c1 雙語教育必學的核心素養－用英文學知識 新視...,99.9
240,5af1c528b44939001e4637b6 玩轉 Unity! 零基礎打造自己的 D ...,99.9
665,6156a77fdf426a0007cc5fe1 建立個人品牌視覺：諾米帶你上手風格再造 T...,99.9
496,5f14aeabcad0d0afe5ea3898 跟 YouTuber 莫彩曦學美國道地的說...,99.9
257,5b31bbf82c4c440043919aae 用手機捕捉瞬息美好-IPPA 冠軍的攝影心...,99.9
203,5add8e08fe4f6d001ee1b60a 空間大數據分析：GIS入門 我們是一個熱愛...,99.9
683,60ed57916444da00061af44d 平面設計概論與流程：李君慈的視覺基礎課 平...,99.9


In [17]:
result['similarity_score'].values[0]

100.0

In [18]:
test_unseen = pd.read_csv('../data/test_unseen.csv')
test_unseen

,user_id,course_id
0,5f1dc5a3afdc0537b5de8979,6156a77fdf426a0007cc5fe1
1,5f432175a32a53813a24e1a5,6156a77fdf426a0007cc5fe1
2,5a93a2b844a1e4001eee7a26,6156a77fdf426a0007cc5fe1
3,617ec17d1a87ca000710ce8f,6156a77fdf426a0007cc5fe1
4,614c3233e64756000603277d,6156a77fdf426a0007cc5fe1
...,...,...
11092,61cf079c1647a30006557fc0,6156a77fdf426a0007cc5fe1
11093,5f54a406f24eac7d75f12cfa,6156a77fdf426a0007cc5fe1
11094,61cf1fec2a965800074634b6,6156a77fdf426a0007cc5fe1
11095,5e8f93b33856b2f1f465aeca,6156a77fdf426a0007cc5fe1


In [19]:
seen_predict_df = pd.read_csv('../data/test_unseen.csv')

In [20]:
userlist = pd.read_csv('../data/users.csv').fillna('')

In [21]:
userlist

,user_id,gender,occupation_titles,interests,recreation_names
0,54ccaa73a784960a00948687,female,,"職場技能_創業,藝術_電腦繪圖,設計_介面設計,設計_動態設計,設計_平面設計,投資理財_投...",
1,54dca4456d7d350900e86bae,male,,"設計_動態設計,設計_平面設計,設計_應用設計,程式_程式入門,程式_程式語言,藝術_角色設...",
2,54e421bac5c9c00900cd8d47,female,,"設計_平面設計,職場技能_資料彙整,藝術_繪畫與插畫,行銷_數位行銷,職場技能_文書處理,職...",
3,54e961d4c5c9c00900cd8d84,other,金融業,"投資理財_理財,攝影_影像創作,投資理財_投資觀念,藝術_更多藝術,音樂_樂器,投資理財_金...",
4,54e9b744c5c9c00900cd8d8a,other,"資訊科技,法律、社會及文化專業,非營利組織","程式_網頁前端,投資理財_理財,投資理財_投資觀念,程式_程式語言,設計_設計理論,投資理財...","政治經濟,社會服務,舞台劇,電影"
...,...,...,...,...,...
130561,62e09de8fc3d3500060d4211,female,,"語言_英文,設計_介面設計,設計_網頁設計,設計_設計理論,程式_軟體程式開發與維護,行銷_...",
130562,62f0823a8c4414000667c592,,,,
130563,631b86242145060007efc7dd,,,,
130564,6331648104ed0f000610dfd2,male,公務人員,"投資理財_理財,攝影_影像創作,攝影_後製剪輯,攝影_商業攝影,投資理財_投資觀念","旅行旅遊,運動健身,金融理財,電影"


In [22]:
group = []
subgroup = []
for interest in userlist['interests']:
    gp = []
    sub = []
    for i in interest.split(','):
        try:
            g, s = i.split('_')
        except:
            continue
        if(g not in gp):
            gp.append(g)
        if(s not in sub):
            sub.append(s)
    group.append(','.join(gp))
    subgroup.append(','.join(sub))
userlist['group'] = group
userlist['subgroup'] = subgroup
userlist

,user_id,gender,occupation_titles,interests,recreation_names,group,subgroup
0,54ccaa73a784960a00948687,female,,"職場技能_創業,藝術_電腦繪圖,設計_介面設計,設計_動態設計,設計_平面設計,投資理財_投...",,"職場技能,藝術,設計,投資理財,行銷","創業,電腦繪圖,介面設計,動態設計,平面設計,投資觀念,數位行銷,角色設計,繪畫與插畫,個人..."
1,54dca4456d7d350900e86bae,male,,"設計_動態設計,設計_平面設計,設計_應用設計,程式_程式入門,程式_程式語言,藝術_角色設...",,"設計,程式,藝術,音樂","動態設計,平面設計,應用設計,程式入門,程式語言,角色設計,介面設計,網頁前端,電腦繪圖,音..."
2,54e421bac5c9c00900cd8d47,female,,"設計_平面設計,職場技能_資料彙整,藝術_繪畫與插畫,行銷_數位行銷,職場技能_文書處理,職...",,"設計,職場技能,藝術,行銷","平面設計,資料彙整,繪畫與插畫,數位行銷,文書處理,職場溝通"
3,54e961d4c5c9c00900cd8d84,other,金融業,"投資理財_理財,攝影_影像創作,投資理財_投資觀念,藝術_更多藝術,音樂_樂器,投資理財_金...",,"投資理財,攝影,藝術,音樂","理財,影像創作,投資觀念,更多藝術,樂器,金融商品,音樂創作"
4,54e9b744c5c9c00900cd8d8a,other,"資訊科技,法律、社會及文化專業,非營利組織","程式_網頁前端,投資理財_理財,投資理財_投資觀念,程式_程式語言,設計_設計理論,投資理財...","政治經濟,社會服務,舞台劇,電影","程式,投資理財,設計","網頁前端,理財,投資觀念,程式語言,設計理論,金融商品"
...,...,...,...,...,...,...,...
130561,62e09de8fc3d3500060d4211,female,,"語言_英文,設計_介面設計,設計_網頁設計,設計_設計理論,程式_軟體程式開發與維護,行銷_...",,"語言,設計,程式,行銷,職場技能","英文,介面設計,網頁設計,設計理論,軟體程式開發與維護,數據分析,日文,應用設計,產品設計,..."
130562,62f0823a8c4414000667c592,,,,,,
130563,631b86242145060007efc7dd,,,,,,
130564,6331648104ed0f000610dfd2,male,公務人員,"投資理財_理財,攝影_影像創作,攝影_後製剪輯,攝影_商業攝影,投資理財_投資觀念","旅行旅遊,運動健身,金融理財,電影","投資理財,攝影","理財,影像創作,後製剪輯,商業攝影,投資觀念"


In [23]:
def clean_data(x):
    return x.replace(" ", "")

fillna=userlist.fillna('')
features= ['occupation_titles', 'interests', 'recreation_names','group','subgroup']
fillna = fillna[features]
for feature in features:
    fillna[feature] = fillna[feature].apply(clean_data)
    
fillna.head(3)

,occupation_titles,interests,recreation_names,group,subgroup
0,,"職場技能_創業,藝術_電腦繪圖,設計_介面設計,設計_動態設計,設計_平面設計,投資理財_投...",,"職場技能,藝術,設計,投資理財,行銷","創業,電腦繪圖,介面設計,動態設計,平面設計,投資觀念,數位行銷,角色設計,繪畫與插畫,個人..."
1,,"設計_動態設計,設計_平面設計,設計_應用設計,程式_程式入門,程式_程式語言,藝術_角色設...",,"設計,程式,藝術,音樂","動態設計,平面設計,應用設計,程式入門,程式語言,角色設計,介面設計,網頁前端,電腦繪圖,音..."
2,,"設計_平面設計,職場技能_資料彙整,藝術_繪畫與插畫,行銷_數位行銷,職場技能_文書處理,職...",,"設計,職場技能,藝術,行銷","平面設計,資料彙整,繪畫與插畫,數位行銷,文書處理,職場溝通"


In [24]:
def create_combination(x):
    occupation_titles = x['occupation_titles']
    interests = x['interests']
    recreation_names = x['recreation_names']
    group = x['group']
    subgroup = x['subgroup']
    text = occupation_titles + interests + recreation_names + group + subgroup
    return text
fillna['combination'] = fillna.apply(create_combination, axis=1)
fillna['user_id'] = userlist['user_id']
fillna['combination'].head(2)

0    職場技能_創業,藝術_電腦繪圖,設計_介面設計,設計_動態設計,設計_平面設計,投資理財_投...
1    設計_動態設計,設計_平面設計,設計_應用設計,程式_程式入門,程式_程式語言,藝術_角色設...
Name: combination, dtype: object

In [25]:
fillna

,occupation_titles,interests,recreation_names,group,subgroup,combination,user_id
0,,"職場技能_創業,藝術_電腦繪圖,設計_介面設計,設計_動態設計,設計_平面設計,投資理財_投...",,"職場技能,藝術,設計,投資理財,行銷","創業,電腦繪圖,介面設計,動態設計,平面設計,投資觀念,數位行銷,角色設計,繪畫與插畫,個人...","職場技能_創業,藝術_電腦繪圖,設計_介面設計,設計_動態設計,設計_平面設計,投資理財_投...",54ccaa73a784960a00948687
1,,"設計_動態設計,設計_平面設計,設計_應用設計,程式_程式入門,程式_程式語言,藝術_角色設...",,"設計,程式,藝術,音樂","動態設計,平面設計,應用設計,程式入門,程式語言,角色設計,介面設計,網頁前端,電腦繪圖,音...","設計_動態設計,設計_平面設計,設計_應用設計,程式_程式入門,程式_程式語言,藝術_角色設...",54dca4456d7d350900e86bae
2,,"設計_平面設計,職場技能_資料彙整,藝術_繪畫與插畫,行銷_數位行銷,職場技能_文書處理,職...",,"設計,職場技能,藝術,行銷","平面設計,資料彙整,繪畫與插畫,數位行銷,文書處理,職場溝通","設計_平面設計,職場技能_資料彙整,藝術_繪畫與插畫,行銷_數位行銷,職場技能_文書處理,職...",54e421bac5c9c00900cd8d47
3,金融業,"投資理財_理財,攝影_影像創作,投資理財_投資觀念,藝術_更多藝術,音樂_樂器,投資理財_金...",,"投資理財,攝影,藝術,音樂","理財,影像創作,投資觀念,更多藝術,樂器,金融商品,音樂創作","金融業投資理財_理財,攝影_影像創作,投資理財_投資觀念,藝術_更多藝術,音樂_樂器,投資理...",54e961d4c5c9c00900cd8d84
4,"資訊科技,法律、社會及文化專業,非營利組織","程式_網頁前端,投資理財_理財,投資理財_投資觀念,程式_程式語言,設計_設計理論,投資理財...","政治經濟,社會服務,舞台劇,電影","程式,投資理財,設計","網頁前端,理財,投資觀念,程式語言,設計理論,金融商品","資訊科技,法律、社會及文化專業,非營利組織程式_網頁前端,投資理財_理財,投資理財_投資觀念...",54e9b744c5c9c00900cd8d8a
...,...,...,...,...,...,...,...
130561,,"語言_英文,設計_介面設計,設計_網頁設計,設計_設計理論,程式_軟體程式開發與維護,行銷_...",,"語言,設計,程式,行銷,職場技能","英文,介面設計,網頁設計,設計理論,軟體程式開發與維護,數據分析,日文,應用設計,產品設計,...","語言_英文,設計_介面設計,設計_網頁設計,設計_設計理論,程式_軟體程式開發與維護,行銷_...",62e09de8fc3d3500060d4211
130562,,,,,,,62f0823a8c4414000667c592
130563,,,,,,,631b86242145060007efc7dd
130564,公務人員,"投資理財_理財,攝影_影像創作,攝影_後製剪輯,攝影_商業攝影,投資理財_投資觀念","旅行旅遊,運動健身,金融理財,電影","投資理財,攝影","理財,影像創作,後製剪輯,商業攝影,投資觀念","公務人員投資理財_理財,攝影_影像創作,攝影_後製剪輯,攝影_商業攝影,投資理財_投資觀念旅...",6331648104ed0f000610dfd2


In [27]:
temp_user = fillna[fillna['user_id'] =='5cf43ce1ea2d0500209d8e67']
print(temp_user)
temp_combination = temp_user['combination']
print(temp_combination.values)
s = temp_combination.values[0]
new_string = re.sub('服務業,自由業', '', s, count = 1)
print(new_string)
r = recommend_law(new_string)
r_list = []
for i in range(50):
        r_list.append(info.at[r.index[i],'course_id'])
        print(r_list)

      occupation_titles                interests recreation_names group  \
23628                    設計_平面設計,設計_介面設計,設計_設計理論                     設計   

             subgroup                              combination  \
23628  平面設計,介面設計,設計理論  設計_平面設計,設計_介面設計,設計_設計理論設計平面設計,介面設計,設計理論   

                        user_id  
23628  5cf43ce1ea2d0500209d8e67  
['設計_平面設計,設計_介面設計,設計_設計理論設計平面設計,介面設計,設計理論']
設計_平面設計,設計_介面設計,設計_設計理論設計平面設計,介面設計,設計理論
['56feb1ac2ba01d09002bbb8e']
['56feb1ac2ba01d09002bbb8e', '57c6960f61d9350700e6cff5']
['56feb1ac2ba01d09002bbb8e', '57c6960f61d9350700e6cff5', '6155cda6d425f500065f5c96']
['56feb1ac2ba01d09002bbb8e', '57c6960f61d9350700e6cff5', '6155cda6d425f500065f5c96', '5f7c209762ad22756c7a1c74']
['56feb1ac2ba01d09002bbb8e', '57c6960f61d9350700e6cff5', '6155cda6d425f500065f5c96', '5f7c209762ad22756c7a1c74', '55e6664ffa223d100058f5d6']
['56feb1ac2ba01d09002bbb8e', '57c6960f61d9350700e6cff5', '6155cda6d425f500065f5c96', '5f7c209762ad22756c7a1c74', '55e6664ffa223d100058f5d6'

In [26]:
from tqdm import tqdm
import csv

In [27]:
train = pd.read_csv('../data/train.csv')
train

,user_id,course_id
0,5bdecbfffec014002166796a,5f194354cad0d086f3ee24cf
1,5fedf958af850a915c86362c,5bfd47782d018e0020e4b0e4 5fc4a352d375951a03cc0...
2,5fd255c43136a460c6f3f930,598053e0acc137070007186a 6031136edd9bc1681035b892
3,5a0bde2aa15b3f001e98429a,5fc5ed671be929537e5283bb 5fc5edae001c9102feab8...
4,5fedf8132a0eb0bfab27882b,5e8e9d3c5a140c3d1e1c5245
...,...,...
59732,6114c700aa04ce00067265a5,61034f3d4ef8b500067afc1c
59733,60e66f29be3e3b0006c4db75,55ae208a7b4d9910001198f2
59734,60e2b05ee742c300072ff5b3,600538ff0cf6c91168243a88
59735,60da0995947dfc0fb61ef296,5edf464fb54d0f59f0e7d96a


In [28]:
bought_times = {}
for co in train['course_id'].to_list():
    for c in co.split(' '):
        try:
            bought_times[c] += 1
        except:
            bought_times[c] = 1
popular = sorted(list(bought_times.items()), key=lambda x:x[1], reverse=True)
df2 = pd.DataFrame(popular, columns=['course_id', 'bought_times'])
df3 = pd.merge(df2, course_df, on=['course_id'])
df3

,course_id,bought_times,course_name,teacher_intro,groups,sub_groups,groups+subgroups,topics,description,will_learn,required_tools,recommended_background,target_group
0,5fc5ee1b08b74a6e3723abd2,8582,҉唐鳳҉數位溝通社：就這樣把你增幅,臺灣的自由軟體程式設計師及政治人物，現任行政院政務委員，曾致力於多項專案的開發維護，是 Ha...,"人文,職場技能","社會科學,職場溝通","人文社會科學,職場技能職場溝通","更多職場溝通,社會科學",怎麼讓即使是沒有投票權的朋友，也可以實質影響公共政策？怎麼善用數位溝通工具，卻不被科技制約？...,唐鳳分享公共議題的溝通技巧，以及她如何設計和運用多樣化的數位溝通方法。瞭解如何通過面對面、線...,原則上不用，不過當然需要一台觀看課程的筆電、平板或任一手機。如果需要紙跟筆做筆記，當然很歡迎準備！,這是一堂輕鬆分享的社課，參與者不需具備專業的背景知識喔！,如果你對於討論公共議題有興趣，或是好奇唐鳳愛用的溝通小技巧，都很適合
1,5f7c210b1de7982fb413a3e9,6154,Today at Apple:和設計師馮宇拆解商業 LOGO 案例,Today at Apple 是由 Apple 精心設計的免費課程，透過攝影、藝術與設計、影...,"藝術,設計","字體設計,平面設計,應用設計","藝術字體設計,設計平面設計,設計應用設計","品牌設計,字體設計",�� 講在課程之前 — 品牌的門面：Logo 在生活當中，我們常常聽到 Logo，不過 Lo...,. 理解 Logo 設計的基本原理. 理解 Logo 設計的必備要素. 理解 Logo 設計...,本課程內容著重觀念建立與培養，同學可以利用紙筆或電腦，方便隨時記下重點。並且也歡迎使用自己常...,無需背景知識,. 設計相關科系學生. 設計產業工作者. 所有對 Logo 設計有興趣的人
2,6030c9cd99e14cc2401e66b9,5961,驅動知識生態系論壇｜Hahow 好學校,Hahow 好學校是成立於 年的新創教育科技公司，以知識技能分享為概念創立的線上課程平台，...,"生活品味,職場技能,藝術,行銷","個人品牌經營,創業,心靈成長與教育,數位行銷,更多生活品味,更多職場技能,求職,社群行銷,角色設計","職場技能個人品牌經營,職場技能創業,生活品味心靈成長與教育,行銷數位行銷,生活品味更多生活品...","個人品牌經營,形象,心靈成長與教育,數位行銷,更多創業,更多教育,更多求職,更多職場技能,社...",Hahow 首場趨勢論壇登場！數位轉型下，知識劃破時空的隔閡，當世上問題已不再只有單一解答，...,將會透過 場專題演講、 場嘉賓對談，讓你作為新興時代的一份子，洞燭機先、幫助自己掌握競爭力...,原則上不用，不過當然需要一台觀看課程的筆電、平板或任一手機。也很歡迎你使用紙筆記下你想記下的...,這是一堂深度分享的論壇活動，參與觀眾不需具備專業的背景知識,科技新創產業、知識內容創作者、欲培養數位轉型思維，或想瞭解如何通過科技，傳播知識及影響力的朋友
3,5f7c209762ad22756c7a1c74,5118,Today at Apple:和攝影師 Ada Lin 用 iPhone 學習專業商品攝影,Today at Apple 是由 Apple 精心設計的免費課程，透過攝影、藝術與設計、影...,攝影,商業攝影,攝影商業攝影,美食攝影,�� 講在課程之前 — 商品攝影的重點當你在行銷品牌或產品時，是否常常想要替商品拍照，卻不知...,從無到有地剖析怎麼開始「商品攝影」這件事，了解如何利用自然的燈光，經過恰當的擺放，以及適當後...,一台可以拍照並簡單後製調色的手機。,無需特殊的背景知識。,任何對商品、美食攝影有興趣的同學都歡迎加入！
4,60cb0a440dabda80019d5f7c,4387,遠距工作力：溝通協作到自我管理,Hahow 好學校是成立於 年的新創教育科技公司，以知識技能分享為概念創立的線上課程平台，...,職場技能,更多職場技能,職場技能更多職場技能,更多職場技能,遠距工作力：溝通協作到自我管理的 WFH 必備指南關於講師周震宇老師澄意文創首席講師、知識轉...,#NAME?,在協作工具單元，需要準備可以使用數位工具 notion 的電腦、平板或任一手機。在舒緩伸展單...,這是一堂輕鬆分享遠距工作小技巧的課程，參與者不需具備專業的背景知識。,這堂課程沒有門檻，無論是否正在遠距居家辦公中，都歡迎加入課程，一起學習喔！
...,...,...,...,...,...,...,...,...,...,...,...,...,...
659,5a531b7d5518aa001e320c8c,1,日本電商代購批貨：銷售獲利心法,【分享所學．傳遞價值．幫助你找回初衷】期盼能啟發你的內在智慧，讓你重新定義「生活」與「工作」...,職場技能,創業,職場技能創業,代購批發,課程緣起Hello，我是 Joanna 老師非常感謝曾上過我第一堂課《免批卡免出國，也能日本...,無論你是網購剁手族、網拍代購家，還是電商代購新手，都能在這堂課中，掌握到日本線上與線下代購批...,一台可以上網的手機、平板或電腦,只要對日本及各國電商代購批貨有興趣者，都非常適合這門課程。但如果想取得日本代購批貨資格，可以...,. 網購剁手族. 網拍代購家. 電商代購新手
660,55b0ba667b4d991000119a92,1,超療癒！唐幼馨的孕婦瑜伽,唐幼馨老師，目前懷孕第二胎。在懷孕期間，她親身體會到孕期的不適，因此，針對孕期、產前分娩、產...,生活品味,運動,生活品味運動,瑜珈,各位準媽媽們大家好，懷孕是一個女人一生當中最辛苦的時候。有時候會水腫、腰酸背痛、精神不濟、失...,,瑜伽墊、瑜伽球、瑜伽繩,,
661,56082612c61d930a00456139,1,一學就會的澎澎甜甜圈精緻吊飾,「我是個很幸運的人，把自己準備好後，機會都會自己來敲門。」出了四本黏土捏塑書，上過些電視台個...,手作,手作小物,手作手作小物,手作小物,大家好！我是美幸老師，這堂課將教導各位將新流行的澎澎劑材料，運用我擅長的手作技巧，設計製作出...,學完這堂課後，學生可以運用自己美感設計獨一無二吊飾及仿真甜品裝飾作品,工具：剪刀，熱熔膠，打火機，尖嘴鉗，微量秤or（刻度杯），免洗杯，調棒（如冰棒棍，免洗筷），...,對手作有興趣者，紓解生活壓力等“”“”,男生，女生，手作有興趣者，需要放鬆心情舒解壓力者。
662,55ae36c87b4d99100011990f,1,火箭準備起飛了 Yoga Basics,Rainie於年開始接觸各種運動，包含拳擊、有氧、瑜珈、皮拉提斯等，然而發現練習瑜珈不但能讓...,生活品味,運動,生活品味運動,瑜珈,火箭瑜珈是傳統Ashtanga的改版，屬於動態且節奏比較快的瑜珈，在多樣化的核心訓練後會進行...,進一步開始練習Rocket與Rocket完整課程,瑜珈墊、牆壁,對瑜珈之呼吸練習、體位法有興趣,想要強化核心肌群、增加肌力和柔軟度的每個人


In [29]:
popular_string = ' '.join([x[0] for x in popular[:50]])

In [30]:
popular_string

'5fc5ee1b08b74a6e3723abd2 5f7c210b1de7982fb413a3e9 6030c9cd99e14cc2401e66b9 5f7c209762ad22756c7a1c74 60cb0a440dabda80019d5f7c 5f7c212262ad2203e77a1cc9 5ef099ab678184065fd4d426 6059aee039f2512548c187c6 5ef0a8a5775dea0566e6cb5f 5ef0a2198e8212c4e6335387 600538ff0cf6c91168243a88 60ac8e42992b7aadf9d019d8 5fc5ed671be929537e5283bb 60b49deac0bb3035f1a6f885 5fd9b1ce0fb8aa8b32928d5b 5fb5fdd083afa949558a7d88 5f14aeabcad0d0afe5ea3898 5e8d70a52aa77e6f34d6fded 5ee4d65789dc7e4854909ba1 5fc4a352d375951a03cc0d45 5fc5edae001c9102feab8ecf 5f94e4a770a38a8694392cf7 604ad46bb0ab47625cd2b2f2 5f5741645077a0553173c948 605c5e82acf40e5c000a7566 60bdd39faa9a66c1691eb2fe 5e55f57bc8bfb600249406e1 5bfd47782d018e0020e4b0e4 5ff825f4d86b7b613942ee70 60c5ce16962ad61a86c7865b 5b26587f67ff51001e25cf0a 58ca97dada45810700e53205 56189df9df7b3d0b005c6639 5ba60a476971da001ea6a46f 607280245075a45ba6da6b5c 5a2170d5a6d4a5001ec3148d 5bdaafa726b5640020754649 5fc745f4df830cebe4964419 5c2465d940e6990020778531 60d801869643fa393b2e209c

In [31]:
predict_users = seen_predict_df["user_id"].to_list()
with open("predict.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["user_id", "course_id"])
    for user_id in tqdm(predict_users):
        #print(user_id)
        temp_user = fillna[fillna['user_id'] == user_id]
        temp_combination = temp_user['combination']
        r_list = []
        s = temp_combination.values[0]
        try:
            r = recommend_law(s)
            if r['similarity_score'].values[0]<=0:
                r_list.append(popular_string)
            else:
                for i in range(50):
                    r_list.append(info.at[r.index[i],'course_id'])
        except:
            try:
                new_string = re.sub('藝術_', '', s)
                r = recommend_law(new_string)
                if r['similarity_score'].values[1]<=0:
                    r_list.append(popular_string)
                else:
                    for i in range(50):
                        r_list.append(info.at[r.index[i],'course_id'])
            except:
                try:
                    new_string = re.sub('服務業,自由業', '', s, count = 1)
                    r = recommend_law(new_string)
                    if r['similarity_score'].values[1]<=0:
                        r_list.append(popular_string)
                    else:
                        for i in range(50):
                            r_list.append(info.at[r.index[i],'course_id'])
                except:
                    r_list.append(popular_string)
                    print(r_list)
                    print(user_id)
                    
        recommend = " ".join(r_list)
        writer.writerow([user_id, recommend])

  0%|          | 18/11097 [00:03<27:16,  6.77it/s] 

['5fc5ee1b08b74a6e3723abd2 5f7c210b1de7982fb413a3e9 6030c9cd99e14cc2401e66b9 5f7c209762ad22756c7a1c74 60cb0a440dabda80019d5f7c 5f7c212262ad2203e77a1cc9 5ef099ab678184065fd4d426 6059aee039f2512548c187c6 5ef0a8a5775dea0566e6cb5f 5ef0a2198e8212c4e6335387 600538ff0cf6c91168243a88 60ac8e42992b7aadf9d019d8 5fc5ed671be929537e5283bb 60b49deac0bb3035f1a6f885 5fd9b1ce0fb8aa8b32928d5b 5fb5fdd083afa949558a7d88 5f14aeabcad0d0afe5ea3898 5e8d70a52aa77e6f34d6fded 5ee4d65789dc7e4854909ba1 5fc4a352d375951a03cc0d45 5fc5edae001c9102feab8ecf 5f94e4a770a38a8694392cf7 604ad46bb0ab47625cd2b2f2 5f5741645077a0553173c948 605c5e82acf40e5c000a7566 60bdd39faa9a66c1691eb2fe 5e55f57bc8bfb600249406e1 5bfd47782d018e0020e4b0e4 5ff825f4d86b7b613942ee70 60c5ce16962ad61a86c7865b 5b26587f67ff51001e25cf0a 58ca97dada45810700e53205 56189df9df7b3d0b005c6639 5ba60a476971da001ea6a46f 607280245075a45ba6da6b5c 5a2170d5a6d4a5001ec3148d 5bdaafa726b5640020754649 5fc745f4df830cebe4964419 5c2465d940e6990020778531 60d801869643fa393b2e209

  2%|▏         | 259/11097 [00:44<23:19,  7.74it/s]  

In [ ]:
#想法 尚未實作:找userid in userlist對應combination 放進recommen_law()